Telecommunications customer churn
====================================

# Exploratory data analysis

Inspired by [this Kaggle dataset](https://www.kaggle.com/datasets/blastchar/telco-customer-churn/data). The actual data I used was obtained from [IBM](https://accelerator.ca.analytics.ibm.com/bi/?perspective=authoring&pathRef=.public_folders%2FIBM%2BAccelerator%2BCatalog%2FContent%2FDAT00148&id=i9710CF25EF75468D95FFFC7D57D45204&objRef=i9710CF25EF75468D95FFFC7D57D45204&action=run&format=HTML&cmPropStr=%7B%22id%22%3A%22i9710CF25EF75468D95FFFC7D57D45204%22%2C%22type%22%3A%22reportView%22%2C%22defaultName%22%3A%22DAT00148%22%2C%22permissions%22%3A%5B%22execute%22%2C%22read%22%2C%22traverse%22%5D%7D), updated 4/2023 (also useful for reference about the columns). 

The Telco customer churn data contains information about a fictional telco company that provided home phone and Internet services to 7043 customers in California in Q3. It indicates which customers have left, stayed, or signed up for their service. 

The churn column indicates whether or not the customer left within the last month. Other columns include gender, dependents, monthly charges, and many with information about the types of services each customer has. Multiple important demographics are included for each customer, as well as a Satisfaction Score, Churn Score, and Customer Lifetime Value (CLTV) index.

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

The advantage of `Path` below is that it comes with cross-platform compatibility

In [30]:
import pandas as pd
import pathlib
import math
import seaborn as sns

## 1. Big picture: Frame the problem



### <span style="background-color: lightblue;">From the business side</span>

#### What is the business objective?💰

- [x] Understand the factors driving customer churn
- [x] Predict:
    - which customers are more likely to churn
    - the reason for churning
- [ ] Come up with our own versions of: (TODO❌)  
    - churn score
    - CLTV
    - Compare these with proprietary values

#### What does the current solution look like? 

IBM Analytics computed churn score (likelihood of customer churning) and CLTV (how valuable the customer is). However, the formulas for computing these scores are proprietary.

### <span style="background-color: lightgreen;">ML specifics</span>

#### What kind of ML method will we use? 

- supervised
- classification
- batch

### <span style="background-color: lightsalmon;">Select a performance measure (cost function)</span>

Accuracy

### Notes📝

#### Intuition

Before looking at the data, let's think about the problem. What could be driving customer churn in a telecom company?

Some of the primary reasons could be:

- competition: better alternative
- unhappy with service: too expensive? bad quality? bad customer service?
- deceased customer? (should be a minor factor)

#### Data-driven questions

- Is churn concentrated in any specific region? If so, is it correlated with specific reasons for leaving?
- Is churn correlated with specific plans and satisfaction scores?
- What customers should be monitored for likelihood of churning?

## 2. Import dataset

### Read files

Plans, dependents, charges

In [5]:
plans=pd.read_excel(pathlib.Path('../data/services.xlsx'))

Churn reason, category and satisfaction score

In [6]:
status=pd.read_excel(pathlib.Path('../data/churn_status.xlsx'))

Location

In [7]:
loc=pd.read_excel(pathlib.Path('../data/location.xlsx'))

### Join

Join all tables into one for convenience

In [8]:
tmp = pd.merge(plans, status, on='Customer ID', how='inner')  # 'inner' join by default
df = pd.merge(tmp, loc, on='Customer ID', how='inner')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 51 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Service ID                         7043 non-null   object 
 1   Customer ID                        7043 non-null   object 
 2   Count_x                            7043 non-null   int64  
 3   Quarter_x                          7043 non-null   object 
 4   Referred a Friend                  7043 non-null   object 
 5   Number of Referrals                7043 non-null   int64  
 6   Tenure in Months                   7043 non-null   int64  
 7   Offer                              3166 non-null   object 
 8   Phone Service                      7043 non-null   object 
 9   Avg Monthly Long Distance Charges  7043 non-null   float64
 10  Multiple Lines                     7043 non-null   object 
 11  Internet Service                   7043 non-null   objec

In [10]:
df.describe()

,Count_x,Number of Referrals,Tenure in Months,Avg Monthly Long Distance Charges,Avg Monthly GB Download,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Count_y,Satisfaction Score,Churn Value,Churn Score,CLTV,Count,Zip Code,Latitude,Longitude
count,7043.0,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.0,7043.000000,7043.000000,7043.000000,7043.000000,7043.0,7043.000000,7043.000000,7043.000000
mean,1.0,1.951867,32.386767,22.958954,20.515405,64.761692,2280.381264,1.962182,6.860713,749.099262,3034.379056,1.0,3.244924,0.265370,58.505040,4400.295755,1.0,93486.071134,36.197455,-119.756684
std,0.0,3.001199,24.542061,15.448113,20.418940,30.090047,2266.220462,7.902614,25.104978,846.660055,2865.204542,0.0,1.201657,0.441561,21.170031,1183.057152,0.0,1856.768045,2.468929,2.154425
min,1.0,0.000000,1.000000,0.000000,0.000000,18.250000,18.800000,0.000000,0.000000,0.000000,21.360000,1.0,1.000000,0.000000,5.000000,2003.000000,1.0,90001.000000,32.555828,-124.301372
25%,1.0,0.000000,9.000000,9.210000,3.000000,35.500000,400.150000,0.000000,0.000000,70.545000,605.610000,1.0,3.000000,0.000000,40.000000,3469.000000,1.0,92101.000000,33.990646,-121.788090
50%,1.0,0.000000,29.000000,22.890000,17.000000,70.350000,1394.550000,0.000000,0.000000,401.440000,2108.640000,1.0,3.000000,0.000000,61.000000,4527.000000,1.0,93518.000000,36.205465,-119.595293
75%,1.0,3.000000,55.000000,36.395000,27.000000,89.850000,3786.600000,0.000000,0.000000,1191.100000,4801.145000,1.0,4.000000,1.000000,75.500000,5380.500000,1.0,95329.000000,38.161321,-117.969795
max,1.0,11.000000,72.000000,49.990000,85.000000,118.750000,8684.800000,49.790000,150.000000,3564.720000,11979.340000,1.0,5.000000,1.000000,96.000000,6500.000000,1.0,96150.000000,41.962127,-114.192901


Show only non-numeric features, including potentially categorical data

In [11]:
df.select_dtypes(exclude=['number']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Service ID              7043 non-null   object
 1   Customer ID             7043 non-null   object
 2   Quarter_x               7043 non-null   object
 3   Referred a Friend       7043 non-null   object
 4   Offer                   3166 non-null   object
 5   Phone Service           7043 non-null   object
 6   Multiple Lines          7043 non-null   object
 7   Internet Service        7043 non-null   object
 8   Internet Type           5517 non-null   object
 9   Online Security         7043 non-null   object
 10  Online Backup           7043 non-null   object
 11  Device Protection Plan  7043 non-null   object
 12  Premium Tech Support    7043 non-null   object
 13  Streaming TV            7043 non-null   object
 14  Streaming Movies        7043 non-null   object
 15  Stre

Only numeric data

In [12]:
df.select_dtypes(include=['number']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Count_x                            7043 non-null   int64  
 1   Number of Referrals                7043 non-null   int64  
 2   Tenure in Months                   7043 non-null   int64  
 3   Avg Monthly Long Distance Charges  7043 non-null   float64
 4   Avg Monthly GB Download            7043 non-null   int64  
 5   Monthly Charge                     7043 non-null   float64
 6   Total Charges                      7043 non-null   float64
 7   Total Refunds                      7043 non-null   float64
 8   Total Extra Data Charges           7043 non-null   int64  
 9   Total Long Distance Charges        7043 non-null   float64
 10  Total Revenue                      7043 non-null   float64
 11  Count_y                            7043 non-null   int64

## 3. Exploration

### Where is churn occurring?

Selects only customers who churned

In [13]:
churn=df[ df['Churn Value'] == 1 ]

What fraction of customers are churning?

In [38]:
len(churn)/len(df)

0.2653698707936959

Gets lat and lon distributions for churning customers using a KDE. KDE stands for kernel density estimator, which amounts to converting a cloud of points into a distribution. Notice that usual KDE estimators are *slow*, so I am using a fast method.

In [14]:
import fastkde
density=fastkde.pdf(churn["Longitude"].to_numpy(), churn["Latitude"].to_numpy())

Gets list of major cities in the world

In [15]:
#import geopandas as gpd
#cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [16]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm
from matplotlib import cm

# Create a figure and set up a Cartopy map projection
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.BORDERS, edgecolor='white')
ax.add_feature(cfeature.STATES, edgecolor='white')

# Filter cities that fall within the map's visible extent
#visible_cities = cities.cx[density.var0[0]:density.var0[-1], density.var1[0]:density.var1[-1]]
#for idx, city in visible_cities.iterrows():
    #ax.plot(city.geometry.x, city.geometry.y, marker='o', color='red', markersize=5, transform=ccrs.PlateCarree())
    #ax.text(city.geometry.x + 0.5, city.geometry.y + 0.5, city['name'], fontsize=9, color='white', transform=ccrs.PlateCarree())

# Plot the colormap 
plt.pcolormesh(density.var0, density.var1, density, cmap='turbo', transform=ccrs.PlateCarree())
plt.colorbar()
plt.title("Churn map")
#plt.show()
plt.savefig('../plots/churn-map.jpg', dpi=300)

### Geography of reasons for churning

What are the reasons for customers churning, as a function of location?

#### Handle the categorical features

In [17]:
churnCat = pd.DataFrame(churn['Churn Category'])

In [18]:
from sklearn.preprocessing import OrdinalEncoder
ordinalEncoder = OrdinalEncoder()
churnCatEnc=ordinalEncoder.fit_transform(churnCat)

In [19]:
churnCatEnc[:8]

array([[1.],
       [1.],
       [1.],
       [2.],
       [4.],
       [1.],
       [3.],
       [2.]])

In [20]:
ordinalEncoder.categories_

[array(['Attitude', 'Competitor', 'Dissatisfaction', 'Other', 'Price'],
       dtype=object)]

#### Map

Plots the geographical distribution of churn according to type

In [21]:
for cat, catText in enumerate(ordinalEncoder.categories_[0]): 
    # selects only the data for the given category
    churnSelect = churn[ churnCatEnc == cat ]
    # kernel density
    density=fastkde.pdf(churnSelect["Longitude"].to_numpy(), churnSelect["Latitude"].to_numpy())

    # Create a figure and set up a Cartopy map projection
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # Add map features
    ax.add_feature(cfeature.BORDERS, edgecolor='white')
    ax.add_feature(cfeature.STATES, edgecolor='white')

    # Filter cities that fall within the map's visible extent
    #visible_cities = cities.cx[density.var0[0]:density.var0[-1], density.var1[0]:density.var1[-1]]
    #for idx, city in visible_cities.iterrows():
        #ax.plot(city.geometry.x, city.geometry.y, marker='o', color='red', markersize=5, transform=ccrs.PlateCarree())
        #ax.text(city.geometry.x + 0.5, city.geometry.y + 0.5, city['name'], fontsize=9, color='white', transform=ccrs.PlateCarree())

    # Plot the colormap 
    plt.pcolormesh(density.var0, density.var1, density, cmap='turbo', transform=ccrs.PlateCarree())
    plt.colorbar()    
    plt.title(catText)
    
    plt.savefig('churn'+catText+ '.jpg', dpi=300)

/Users/nemmen/anaconda3/lib/python3.11/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/nemmen/anaconda3/lib/python3.11/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/nemmen/anaconda3/lib/python3.11/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/nemmen/anaconda3/lib/python3.11/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/nemmen/anaconda3/lib/python3.11/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/Users/nemmen/anaconda3/lib/python3.11/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountere

### Correlations with churn

#### Convert categorical values to numbers

Which features to convert?

services/Referred a Friend
services/Phone Service
services/Multiple Lines
services/Internet Service
services/Online Security
services/Online Backup
services/Streaming TV
services/Streaming Movies
services/Streaming Music
services/Contract

In [22]:
ordinalEncoder = OrdinalEncoder()

# List of categorical feature columns
categorical_features = [
    'Referred a Friend',
    'Phone Service',
    'Multiple Lines',
    'Internet Service',
    'Online Security',
    'Online Backup',
    'Streaming TV',
    'Streaming Movies',
    'Streaming Music',
    'Contract',
    'Device Protection Plan',
    'Premium Tech Support',
    'Unlimited Data'
]

# Apply OrdinalEncoder to the categorical columns in the DataFrame
df[categorical_features] = ordinalEncoder.fit_transform(df[categorical_features])

#### Correlation matrix

In [23]:
removeCols= ['Churn Score', 'Count_x', 'Count_y', 'Count', 'CLTV', 'Latitude', 'Longitude', 'Zip Code']
corrMatrix = df.drop(columns=removeCols).corr(method='pearson', numeric_only=True)

In [24]:
corrMatrix["Churn Value"].sort_values(ascending=False, key=abs)

Churn Value                          1.000000
Satisfaction Score                  -0.754649
Contract                            -0.435398
Tenure in Months                    -0.352861
Number of Referrals                 -0.286540
Internet Service                     0.227890
Total Long Distance Charges         -0.223756
Total Revenue                       -0.223003
Total Charges                       -0.198546
Monthly Charge                       0.193356
Online Security                     -0.171226
Unlimited Data                       0.166545
Premium Tech Support                -0.164674
Referred a Friend                   -0.149122
Online Backup                       -0.082255
Device Protection Plan              -0.066160
Streaming TV                         0.063228
Streaming Movies                     0.061382
Avg Monthly GB Download              0.048868
Streaming Music                      0.045587
Multiple Lines                       0.040102
Total Refunds                     

##### Conclusions:

I highlighted below non-obvious correlations with churning. We could devise specific campaigns to retain such customers.

Churn is highly *anti-correlated* with:

1. satisfaction score
2. <u>Type of contract</u>
3. tenure duration

Churn is mildly *correlated* with:

1. internet service
2. monthly charge

Regarding specific plans and options, the ones that seem to affect churning more strongly are:

1. internet
2. <u>online security</u>
3. online backup
4. streaming services (similar impact)

#### Logistic regression coefficients

I fit a Logistic Regression model where the target variable is binary (`Churn Value`). The coefficients of the logistic regression model help us understand how each numerical feature impacts the odds of churn. The strength and direction of the relationship between each numerical feature and the binary Churn variable.

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [26]:
# Assuming df is your DataFrame and 'Churn' is the binary target variable
remove=['Churn Value']+removeCols
# include only numeric features
X = df.drop(columns=remove).select_dtypes(include=[np.number])
y = df['Churn Value']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Fit the Logistic Regression model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Get the coefficients
coefficients = pd.Series(log_reg.coef_[0], index=X.columns)

In [27]:
coefficients.sort_values(ascending=False, key=abs)

Satisfaction Score                  -7.130723
Number of Referrals                 -2.049772
Online Security                     -1.369833
Tenure in Months                    -1.210447
Monthly Charge                       0.794078
Referred a Friend                    0.779304
Contract                            -0.618929
Premium Tech Support                -0.280887
Phone Service                       -0.269804
Internet Service                     0.250590
Total Charges                        0.243089
Total Revenue                        0.242088
Multiple Lines                       0.198481
Avg Monthly Long Distance Charges   -0.195545
Total Refunds                       -0.181187
Online Backup                       -0.171881
Total Long Distance Charges          0.168446
Unlimited Data                      -0.161912
Avg Monthly GB Download             -0.101038
Streaming Movies                     0.067514
Total Extra Data Charges            -0.052217
Device Protection Plan            

##### Conclusions 2:

A quick logistic regression analysis further reveals interesting correlations. New results compared to the previous correlation analysis are highlighted.

Churn highly *anti-correlated* with:

1. satisfaction score
2. <u>Number of referrals</u>
3. Online security
4. tenure duration
5. type of contract

Churn *correlated* with:

1. monthly charge 
2. referred a friend (correlated with 2 above)
3. total charges (correlated with `a`)

Another surprise is that logistic regression suggests that the factors below are not as relevant as those pointed out above: 

1. online backup
2. internet service
3. streaming services (similar impact)

### Visual inspection

#### Boxplots

In [31]:
# Set up the 5x5 grid
n_cols = 5
n_rows = math.ceil(len(X.columns) / n_cols)  # Dynamically calculate rows needed

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 15))  # Adjust size of the plot grid
axes = axes.flatten()  # Flatten the axes to easily index in the loop

# Plot boxplots for each column in the grid
for i, column in enumerate(X.columns):
    sns.boxplot(x='Churn Value', y=column, data=df, ax=axes[i], hue='Churn Value', legend=False)
    axes[i].set_title(f'{column} vs Churn Value')

# Remove unused subplots if there are fewer plots than grid cells
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.savefig('../plots/boxplot.png', dpi=300)

#### Violin plot

In [32]:
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 15))  # Adjust size of the plot grid
axes = axes.flatten()  # Flatten the axes to easily index in the loop

# Plot boxplots for each column in the grid
for i, column in enumerate(X.columns):
    sns.violinplot(x='Churn Value', y=column, data=df, ax=axes[i], hue='Churn Value', legend=False)
    axes[i].set_title(f'{column} vs Churn Value')

# Remove unused subplots if there are fewer plots than grid cells
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.savefig('../plots/violin.png', dpi=300)

I organized these plots, removing unnecessary panels in `plots.key`.

# Sandbox

In [37]:
len(churn)/len(df)

0.2653698707936959

In [36]:
len(df)

7043